In [ ]:
%run src/IotNodes.ipynb
%run src/BaseStation.ipynb
%run src/Agent.ipynb
%run src/Block.ipynb

In [ ]:
#import imp
#from pkgutil import ImpImporter

import random
import pickle
import networkx as nx

from numpy import block
from configparser import ConfigParser
  
configur = ConfigParser()

import builtins
configur.read(builtins.current_filename)
print(builtins.current_filename)
#configur.read('config.ini')

defTtl is the default initial total time to live for each package obtained from the configuration file.

In [ ]:
defTtl = int(configur.get('packet','def_ttl'))
# generate map of size n*m 
## p is probability of getting a UAV at particular cell

We are defining a Map class, where the Map is initialiased with a grid of self.n rows and self.m columns, where the probability is self.p.
In case of a random map, p is the probability that a cell would have a UAV (It would have an IoT node with prob. 1-p)
self.map is a matrix storing the networking device in each cell
self.agents = [] empty list of UAV nodes
self.Iot_Nodes = [] empty list of Iot_Nodes
self.BaseStation = None BaseStation.

In [1]:
class Map():
    def __init__(self,n,m,p):
        self.n = self.row = n 
        self.m = self.col = m 
        self.p=p
        # self.map = []
        # self.agents = []
        # self.Iot_Nodes = []
        # self.BaseStation = None

        self.map = [['-' for i in range(self.col)] for j in range(self.row)]
        self.Agents = []
        self.IotNodes = []
        self.BaseStation = None

        self.G = nx.Graph()

        self.defTtl = 1000
        self.iot_rate = 20

read
- read the map from the file and transfer it to a map matrix.
- print the name of the file just read
- Set the initial rate of package generation, for IotNodes.generatePacket()
- i is row, and j is col.
- loop through each character in the file.
    - is the char is empty, continue
    - if char == 'B', then set map_ij to BaseStation object, and assign it to self.BaseStation
    - if char == 'A', then create a UV node, with no neighbours, and temporary BaseStation, and assign it to map_ij, and append it to agents list
    - if char == 'I', then it is a packet generating IotNode, create an IotNode object and assign it to map_ij and append it to Iot_Nodes list.
    - if char == 'X', it is a block of no value, create a block object and assign it to map_ij.
 
- again loop through each row and each column, to add neighbours and BaseStation to UAV agents, what about transmission distance - Doubt - Pending
    - if the cell is a base or block continue.
    - if it is a UAV, update its basestation.
    - if NEWS(north, east, west, south) is a UAV or BaseStation, append it as a neighbour.
 
- assign map_ to self.map 

In [ ]:
# class Map(Map):
#     def read(self):
#         file  = open('./Maps/'+configur.get('map','name'),'r') 
#         map_=[['-' for i in range(self.m)] for j in range(self.n)]
#         print("read ",file)
#         i=0
#         j=0
#         initial_rate=10
#         for line in file:
#             j=0
#             for char in line:
#                 if char==' ':
#                     continue
#                 if char=='B':
#                     map_[i][j]= BaseStation(i,j)
#                     self.BaseStation = map_[i][j]
#                 elif char == 'A':
#                     agent  = Agent([], i, j, self.BaseStation)
#                     map_[i][j]= agent
#                     self.agents.append(agent)
#                 elif char == 'I':
#                     #rate=  random.randint(2,10) 
#                     rate = initial_rate # with uniform random generation of packets, avergae will be 7 (<transmission rate)
#                     #initial_rate += 1
#                     iot = IotNodes(rate, defTtl,i,j)
#                     map_[i][j]= iot
#                     self.Iot_Nodes.append(iot)
#                 elif char=='X':
#                     block= Block((i,j))
#                     map_[i][j]=block
#                 j+=1
#                 if(j==self.m): break
#             i+=1
#         for i in range(self.n):
#                 for j in range(self.m):
#                     if map_[i][j].isBase() or map_[i][j].isBlock():
#                         continue
#                     if(map_[i][j].isUAV()):
#                         map_[i][j].targetBaseStation = self.BaseStation
#                     # if map_[i,j].isUAV: # commenting this because IoT nodes also need neighbours
#                     if i>0 and (map_[i-1][j].isUAV() or map_[i-1][j].isBase()):
#                         map_[i][j].addNeighbour(map_[i-1][j])
#                     if j>0 and (map_[i][j-1].isUAV() or map_[i][j-1].isBase()):
#                         map_[i][j].addNeighbour(map_[i][j-1])
#                     if i<self.n-1 and (map_[i+1][j].isUAV() or map_[i+1][j].isBase()):
#                         map_[i][j].addNeighbour(map_[i+1][j])
#                     if j<self.m-1 and (map_[i][j+1].isUAV() or map_[i][j+1].isBase()):
#                         map_[i][j].addNeighbour(map_[i][j+1])

#         self.map = map_ 


In [ ]:
class Map(Map):
    
    def node_to_matrix(self, node):
        r = node//self.col
        c = node%self.col
        
        return [r,c]
        
    def cell_to_node(self, pos):
        r, c = pos[0], pos[1]
        node = r*self.col + c

        return node
    
    def read(self, given_map):

        self.G.add_nodes_from(given_map['nodes'])
        self.G.add_edges_from(given_map['edges'])
        
        for node in given_map['BaseStation']:
            x,y = self.node_to_matrix(node)
            self.BaseStation = BaseStation(x,y)
            self.map[x][y] = self.BaseStation
        
        for node in given_map['Agents']:
            x,y = self.node_to_matrix(node)
        
            agent = Agent([], x, y, self.BaseStation)
            self.map[x][y] = agent
            self.Agents.append(agent)

        for node in given_map['Agents']:
            x,y = self.node_to_matrix(node)
        
            node_neighbours = [edge[1] for edge in self.G.edges(node) if ((edge[1] in given_map['Agents']) or (edge[1] in given_map['BaseStation']))]

            for neigh in node_neighbours:
                a,b = self.node_to_matrix(neigh)
                self.map[x][y].addNeighbour(self.map[a][b])

        for node in given_map['IotNodes']:
            x,y = self.node_to_matrix(node)
            iot_neighbours = [edge[1] for edge in self.G.edges(node) if ((edge[1] in given_map['Agents']) or (edge[1] in given_map['BaseStation']))]
            
            iot = IotNodes(self.iot_rate, self.defTtl,x,y)
        
            for neigh in iot_neighbours:
                a,b = self.node_to_matrix(neigh)
                iot.addNeighbour(self.map[a][b])
            
            self.map[x][y] = iot
            self.IotNodes.append(iot)
        
        for node in given_map['Blocks']:
            x,y = self.node_to_matrix(node)
            
            block= Block((x,y))
            self.map[x][y] = block
            
    def from_file_graph(self):
        file  = open('./Maps/'+configur.get('map','name'),'rb') 
        self.map_info_dict = pickle.load(file)
        file.close()
        self.read(self.map_info_dict)

- generate
    - this function generates a map
    - map_ is matrix with same rows and columns as the map
    - x and y - are randomly chosen as the location of the BaseStation.
        - This BaseStation is stored in map_xy, and self.BaseStation
    - run a loop through the rows
        - run a loop through the columns
            - if the location is the basestation, continue
            - prob is randomly chosen
            - if the prob < self.p(prob of UAV generation)
                - generate a UAV, store it in the map, and append it to agents
            - else:
                - generate an IotNode, store it in the map, and append it to Iot_Nodes.
                
- again loop through each row and each column, to add neighbours and BaseStation to UAV agents, what about transmission distance - Doubt - Pending
    - if the cell is a base or block continue.
    - if it is a UAV, update its basestation.
    - if NEWS(north, east, west, south) is a UAV or BaseStation, append it as a neighbour.
 
- assign map_ to self.map 

In [ ]:
# class Map(Map):
#     def generate(self):
#         map_=[['-' for i in range(self.m)] for j in range(self.n)]
#         x=random.randint(0,self.n-1)
#         y=random.randint(0,self.m-1)
#         map_[x][y]= BaseStation(x,y)
#         self.BaseStation = map_[x][y]
#         for i in range(self.n):
#             for j in range(self.m):
#                 if i==x and j==y:
#                     continue
#                 prob=random.uniform(0,1)
#                 if(prob<=self.p):
#                     agent  = Agent([], i, j, self.BaseStation)
#                     map_[i][j]= agent
#                     self.agents.append(agent)
#                 else:
#                     # rate=  random.randint(0,10) # TODO: add actual rate
#                     rate = 10 # with uniform random generation of packets, avergae will be 7 (<transmission rate)
#                     iot = IotNodes(rate, defTtl,i,j) 
#                     map_[i][j]= iot
#                     self.Iot_Nodes.append(iot)  

#         # populate neightbours for each agent
#         for i in range(self.n):
#             for j in range(self.m):
#                 if map_[i][j].isBase():
#                     continue
#                 # if map_[i,j].isUAV: # commenting this because IoT nodes also need neighbours
#                 if i>0 and (map_[i-1][j].isUAV() or map_[i-1][j].isBase()):
#                     map_[i][j].addNeighbour(map_[i-1][j])
#                 if j>0 and (map_[i][j-1].isUAV() or map_[i][j-1].isBase()):
#                     map_[i][j].addNeighbour(map_[i][j-1])
#                 if i<self.n-1 and (map_[i+1][j].isUAV() or map_[i+1][j].isBase()):
#                     map_[i][j].addNeighbour(map_[i+1][j])
#                 if j<self.m-1 and (map_[i][j+1].isUAV() or map_[i][j+1].isBase()):
#                     map_[i][j].addNeighbour(map_[i][j+1])

#         self.map = map_ 
#         # print("map created of length: ", len(self.map))

getBaseStation - return the BaseStation
getAgents - get the list of all the UAV Nodes
getIotNodes - get the list of all the IotNodes

In [ ]:
class Map(Map):
    def getBaseStation(self):
        return self.BaseStation
class Map(Map):    
    def getAgents(self):
        return self.Agents
class Map(Map):     
    def getIotNodes(self):
        return self.IotNodes 

This function basically prints the map.
X - block
|| - basestation
- - - IoTNodes
rest are UAV's
Make it more readable and understandable.

In [ ]:
class Map(Map):
    def renderMap(self):
        for i in range(self.m):
            print('-'*6,end="")
        print()
        for i in range(self.n):
            print('|',end="")
            for j in range(self.m):
                if self.map[i][j].isBlock():
                    print(' |',end="")
                    print('X',end="")
                    print('| ',end="")
                elif self.map[i][j].isBase():
                    print(' |',end="")
                    print(self.map[i][j].getVal(),end="")
                    print('| ',end="")
                elif self.map[i][j].isIot():
                    print(' -',end="")
                    print(len(self.map[i][j].queue),end="")
                    print('- ',end="")
                else:
                    print('  ',end="")
                    print(self.map[i][j].getVal(),end="")
                    print('  ',end="")
                print('|',end="")
            print()
            for j in range(self.m):
                print('-'*6,end="")
            print()
        print()

This creates a dummy map, which is - 
Base UAV IoT

In [ ]:
class Map(Map):
    def dummyMap(self):  ## 1*3 map
        map_= map_=[['-' for i in range(3)] for j in range(1)]
        map_[0][0] = BaseStation(0,0)
        self.BaseStation = map_[0][0]

        agent  = Agent([], 0, 1, self.BaseStation)
        map_[0][1]= agent
        self.agents.append(agent)

        #rate=  random.randint(0,10) # TODO: add actual rate
        rate = 10
        iot = IotNodes(rate, defTtl,0,2) 
        map_[0][2]= iot
        self.Iot_Nodes.append(iot)  


        map_[0][1].addNeighbour(map_[0][0])
        map_[0][2].addNeighbour(map_[0][1])
        self.map = map_ 

This function resets each object in the cell of the map, be it a block, UAV, Iot or Base Station

In [ ]:
class Map(Map):
    def resetAll(self):
        for i in range(self.n):
            for j in range(self.m):
                self.map[i][j].reset()

This function loads the unique model for each UAV agent

In [ ]:
class Map(Map):
    def loadModel(self,foldername):
        for i in range(self.n):
            for j in range(self.m):
                if(self.map[i][j].isUAV()):
                    self.map[i][j].loadModel("./{}/agent_at_{}".format(foldername,(i,j)))

This function summons the function to create a DQNAgent object for each UAV

In [ ]:
class Map(Map):   
    def initModels (self,device):
        for agent in self.Agents:
            agent.initDQN(device)